In [1]:
import requests
import json
import time

In [2]:
with open('C:/Users/Dominika/Downloads/low_supp_1019_berkley.json', 'r') as file:
    bodyTemplate = json.load(file)

In [3]:
with open('support_for_berkley.json', 'r') as file:
    top40 = json.load(file)

In [4]:
bodyTemplate[2]["parameters"]["min"]

50

In [5]:
row_id = 0

for row in top40:
    row_id+=1

    bodyTemplate[1]["parameters"]["patterns"][0]["head"]["object"]["value"] = "<"+row+">"
    bodyTemplate[1]["parameters"]["thresholds"][0]["value"] = int(top40[row]*0.1)
    bodyTemplate[2]["parameters"]["min"] = int(top40[row]*0.1)

    URL = "http://rdfrules.vse.cz/api/task"

    body = bodyTemplate

    r = requests.post(url = URL, json = body)

    data = r.json()

    id = data['id']
    print("Odeslal se pozadavek na nalezeni pravidel k mediu ", row, ", task id je ", id, "support je ", int(top40[row]*0.1))
    print(bodyTemplate)

    
    #get requests
    getURL = "http://rdfrules.vse.cz/api/task/"+id


    r = requests.get(getURL)

    time.sleep(15)

    while r.status_code == 202:
        r = requests.get(getURL)
        print(row, "(", row_id, "/", len(top40), ")", r.status_code," ... LOGY:", r.json()['logs'])
        time.sleep(15)
    else:
        if r.status_code == 200:
            print("\n počet nalezených pravidel k mediu ", row, "(", row_id, "/", len(top40), ")", ": ", len(r.json()['result']), "\n")
        elif r.status_code != 202 and r.status_code != 200:
            print("\n v mediu ", row, " task ", id, " nastala chyba ", r.status_code, "\n")

    for row in r.json()['result']:
        f = open("C:/Users/Dominika/Documents/SKOLA/ING/DIPLOMKA/output/ruleset_low_support_berkley.ndjson", "a")
        f.write(str(row)+"\n")
        f.close()




Odeslal se pozadavek na nalezeni pravidel k mediu  urn:unknown:medium%3A65 , task id je  19974de4-a7e3-441e-8989-29532c28b89a support je  172
[{'name': 'LoadIndex', 'parameters': {'path': '/kg-microbe/train_data_1019.index', 'partially': False}}, {'name': 'Mine', 'parameters': {'thresholds': [{'name': 'MinSupport', 'value': 172}, {'name': 'MaxRuleLength', 'value': 4}, {'name': 'LocalTimeout', 'value': 10000, 'me': 0.2, 'dme': False}], 'ruleConsumers': [{'name': 'topK', 'k': 100000, 'allowOverflow': True}], 'patterns': [{'body': [], 'head': {'subject': {'name': 'Variable', 'value': 'a'}, 'predicate': {'name': 'Constant', 'value': 'biolink:occurs_in'}, 'object': {'name': 'Constant', 'value': '<urn:unknown:medium%3A65>'}, 'graph': {'name': 'Any'}}, 'exact': False, 'orderless': True}], 'constraints': [{'name': 'WithoutPredicates', 'values': ['<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>', 'rdfs:label']}, {'name': 'OnlyLowerCardinalitySideConstants'}, {'name': 'WithoutDuplicitPredicate

In [88]:
rules = []
with open("C:/Users/Dominika/Documents/SKOLA/ING/DIPLOMKA/output/ruleset.ndjson") as file:
    for line in file: 
        line = line.strip() #or some other preprocessing
        rules.append(line) #storing everything in memory!

In [90]:
len(rules)

548

In [96]:
for rule in rules:
    objects = json.loads(rule)["body"]
    for object in objects:
        if (isinstance(objects[objects.index(object)]["object"]["value"], dict)):
            if (objects[objects.index(object)]["object"]["value"]["localName"] == "OrganismTaxon"):
                rules.pop(rules.index(rule))

len(rules)

218

In [98]:
for row in rules:
    f = open("C:/Users/Dominika/Documents/SKOLA/ING/DIPLOMKA/output/ruleset_only_good.ndjson", "a")
    f.write(str(row)+"\n")
    f.close()